In [1]:
import pandas as pd
import numpy as np
from pydriller import Repository

In [4]:
def getChangesInRepository(repoPath):
    changes = []

    for commit in Repository(repoPath).traverse_commits():
        for file in commit.modified_files:
            record = {
                'file': file.filename,
                'lines_added': file.added_lines,
                'lines_deleted': file.deleted_lines,
                'commit': commit.hash,
                'author': commit.author.name
            }
            changes.append(record)

    return changes

In [5]:
changes = getChangesInRepository('https://github.com/IntegerMan/GitVisualizer')
changes

[{'file': '.gitignore',
  'lines_added': 0,
  'lines_deleted': 0,
  'commit': 'a85288e6deee374264b13d3143c886962e90550d',
  'author': 'Matt Eland'},
 {'file': 'README.md',
  'lines_added': 1,
  'lines_deleted': 0,
  'commit': 'a85288e6deee374264b13d3143c886962e90550d',
  'author': 'Matt Eland'},
 {'file': '.gitignore',
  'lines_added': 139,
  'lines_deleted': 0,
  'commit': 'c58bad1219a1cf9b9b1105791fffc331a6d902fe',
  'author': 'Matt Eland'},
 {'file': 'Experimentation.ipynb',
  'lines_added': 36,
  'lines_deleted': 0,
  'commit': 'c58bad1219a1cf9b9b1105791fffc331a6d902fe',
  'author': 'Matt Eland'}]